In [65]:
import random
import pandas as pd

In [66]:
class Product:
    def __init__(self, id, expiry, value, quantity, incompatible=[]):
        self.id = id  
        self.expiry = expiry  
        self.value = value  
        self.quantity = quantity  
        self.incompatible = incompatible 

    def can_ship_together(self, prod2):
        return prod2.id not in self.incompatible and self.id not in prod2.incompatible

    def __str__(self):
        return (f"Product(ID: {self.id}, Expiry: {self.expiry}, "
                f"Value: {self.value}, Quantity: {self.quantity}, "
                f"Incompatible: {self.incompatible})")


In [67]:
def display_products_table(products):
    products_data = [{
        "ID": product.id,
        "Expiry": product.expiry,
        "Value": product.value,
        "Quantity": product.quantity,
        "Incompatible": ", ".join(map(str, product.incompatible))
    } for product in products]
    
    df = pd.DataFrame(products_data)
    
    
    display(df)  

In [68]:
def init():
    products = []
    MAX_TRIPS = None
    population_size = None      
    parent_ratio = None           
    elite_ratio = None           
    mutation_rate = None        
    inheritance_ratio = None          
    generations = None
    
    
    # if input("Wanna use default products? (y/n): ") != 'n':
    if 1 :

        population_size = 50          
        parent_ratio = 0.4             
        elite_ratio = 0.2              
        mutation_rate = 0.01                    
        generations = 10  
        inheritance_ratio = 0.7
        

        products = [
            Product(1, 3, 15, 4),
            Product(2, 6, 10, 2),
            Product(3, 5, 30, 10),
            Product(4, 4, 25, 4),
            Product(5, 9, 20, 3),
            Product(6, 1, 50, 4),
            Product(7, 7, 35, 5),
            Product(8, 8, 25, 1),
            Product(9, 5, 50, 3),
            Product(10, 5, 20, 3)
        ]
        
        
        incompatible = {
            1: [1, 10],
            2: [4, 5],
            4: [7],
            5: [6],
            6: [1],
            7: [5, 10],
            8: [1, 8],
            9: [2, 3],
            10: [1, 5]
        }
        
        MAX_TRIPS = 15  # one-way


        for p in products:
            if p.id in incompatible:
                p.incompatible = incompatible[p.id]


    # else:
    #     print("so what do you want to do?")
    #     while(1):
    #         p = list(map(int, input("Enter product (id expiry value quantity) in one line: ").split()))
    #         if p == []:
    #             break
    #         if len(p) != 4:
    #             print("Invalid input")
    #             continue
    #         products.append(Product(p[0], p[1], p[2], p[3]))

    #     for p in products:
    #         print(p)

    #     for p in products:
    #         p.incompatible = list(
    #             map(int, input(f"Enter incompatible products for {p}: ").split()))
            
    #     MAX_TRIPS = int(input("Enter max trips: "))
    #     population_size = int(input("Enter population size: "))
    #     parent_ratio = float(input("Enter parent ratio: "))
    #     elite_ratio = float(input("Enter elite ratio: "))
    #     mutation_rate = float(input("Enter mutation rate: "))
    #     inheritance_ratio = float(input("Enter inheritance_ratio: "))
    #     generations = int(input("Enter number of generations: "))


    
    return products, MAX_TRIPS, population_size, parent_ratio, elite_ratio, mutation_rate, inheritance_ratio, generations


In [69]:
   
class GeneticAlgorithm():
    def __init__(self, products, MAX_TRIPS , population_size=50, 
                 parent_ratio=0.4, elite_ratio=0.2, mutation_rate=0.001, inheritance_ratio=0.7, generations=10):
        
        self.products = products
        self.MAX_TRIPS = MAX_TRIPS
        self.population_size = population_size
        self.parent_ratio = parent_ratio
        self.elite_ratio = elite_ratio
        self.mutation_rate = mutation_rate
        self.inheritance_ratio = inheritance_ratio
        self.generations = generations
        self.best_solution = None
        self.best_fitness = float('-inf')
        self.best_generation = None
        

    def generate_chromosome(self):
        chromosome = []
        available_products = []
        for product in self.products:
            available_products.extend([product] * product.quantity)
    
        random.shuffle(available_products)
    
        for trip in range(self.MAX_TRIPS):
            
            valid_products = [p for p in available_products if p.expiry > trip]
            if not valid_products:
                chromosome.append(tuple())
                continue
            
        
            prod1 = random.choice(valid_products)
            available_products.remove(prod1)
            valid_products.remove(prod1)
        
            
            valid_companions = [p for p in valid_products if prod1.can_ship_together(p)]
        
            if valid_companions:
                prod2 = random.choice(valid_companions)
                available_products.remove(prod2)
                valid_products.remove(prod2)
                chromosome.append((prod1.id, prod2.id))
            else:
                chromosome.append((prod1.id,))
    
        return chromosome

    def generate_population(self):
        return [self.generate_chromosome() for _ in range(self.population_size)]
    
    def get_chromosome_products(self, chromosome):
        product_dict = {p.id: p for p in self.products}
        chromosome_products = []
        for trip in chromosome:
            trip_products = [product_dict[prod_id] for prod_id in trip]
            chromosome_products.append(trip_products)
    
        return chromosome_products
    
    def is_valid_chromosome(self, chromosome):
    
        delivered_counts = {p.id: 0 for p in self.products}
        chromosome_products = self.get_chromosome_products(chromosome)
    
        for i, trip in enumerate(chromosome_products):
            
            # if len(trip) > 2:
            #     return False

            for j, prod in enumerate(trip):
                if prod.expiry <= i:
                    return False
            
                
                delivered_counts[prod.id] += 1
                if delivered_counts[prod.id] > prod.quantity:
                    return False

                if j == 1:  
                    if not trip[0].can_ship_together(prod):
                        return False

        return True
    
    def fitness(self, chromosome):
        if not self.is_valid_chromosome(chromosome):
            return float('-inf') 
    
        available_products = []
        for product in self.products:
            available_products.extend([product] * product.quantity)
    
        total_value = 0
        for trip in self.get_chromosome_products(chromosome):
            for prod in trip:
                available_products.remove(prod)
                total_value += prod.value

        for p in available_products: 
            if p.expiry <= self.MAX_TRIPS:
                total_value -= p.value
    
        return total_value
    
    def selection(self, population, elite_ratio=0.2):
        # elitism + roulette
        fitnesses = [self.fitness(chromosome) for chromosome in population]

        population_size = len(population)
        elite_size = int(population_size * elite_ratio)

        population_fitness_pairs = list(zip(list(range(len(fitnesses))), fitnesses , population))
        population_fitness_pairs.sort(key=lambda i: i[1], reverse=True)

        elites = population_fitness_pairs[:elite_size]
        available = population_fitness_pairs[elite_size:]

        selected_parents = [chrom for _, _, chrom in elites] 

        available_fitnesses = [f for _, f, _ in available]
        min_fitness = min(available_fitnesses)
        shifted_fitnesses = [f - min_fitness + 1 for f in available_fitnesses]
        total_fitness = sum(shifted_fitnesses)
        available_probs = [f/total_fitness for f in shifted_fitnesses]
    

        available = list(zip(available_probs, available)) # (prob, (idx, fitness, chromosome))
        
        parents_size = int(population_size * self.parent_ratio)
        remaining_size = parents_size - elite_size
        #Select remaining parents using roulette wheel
        for _ in range(remaining_size):
            if not available:
                break

            r = random.random()  # Selection Marker of roulette wheel
            cumsum = 0
            selected = False
            selected_idx = None
            for i, (prob, (idx, fitness, chromosome)) in enumerate(available):
                cumsum += prob
                if cumsum > r:
                    selected_parents.append(chromosome)  
                    selected = True
                    selected_idx = i
                    break

            if selected:
                available.pop(selected_idx)

            if not selected and available:
                selected_tuple = random.choice(available)
                selected_parents.append(selected_tuple[1][2])
                available.remove(selected_tuple)

        return selected_parents
     
    
    def analyze_population(self, population):
        pd.set_option('display.max_rows', None)  
        pd.set_option('display.max_columns', None)  
        pd.set_option('display.width', None)  
        pd.set_option('display.max_colwidth', None)
        table_data = []  
        for idx, chromosome in enumerate(population):
            row = {
                'chromosome': str(chromosome),
                'Fitness': self.fitness(chromosome),
                'Valid': self.is_valid_chromosome(chromosome),
                'Trips Used': len([trip for trip in chromosome if trip])
            }
            
            
            delivered = {p.id: 0 for p in self.products}
            for trip in chromosome:
                for prod_id in trip:
                    delivered[prod_id] += 1
                    
            
            for prod in self.products:
                row[f'P{prod.id}%'] = f'{(delivered[prod.id]/prod.quantity)*100:.0f}%'
            
            
            table_data.append(row)
        
        
        df = pd.DataFrame(table_data)
        
        
        summary = {
            'chromosome': 'Summary',
            'Fitness': f'Best: {df["Fitness"].max():.0f}',
            'Valid': f'Valid: {df["Valid"].sum()}/{len(df)}',
            'Trips Used': f'Avg: {df["Trips Used"].mean():.1f}'
        }
        
        
        df = pd.concat([df, pd.DataFrame([summary])])
        
        
        display(df)

    
    def create_child(self, stronger_parent, weaker_parent, inheritance_ratio):
        child = []

        for gene_idx in range(len(stronger_parent)):
            r = random.random()
            if r < inheritance_ratio:
                child.append(stronger_parent[gene_idx])
            else:
                child.append(weaker_parent[gene_idx])

        return tuple(child)
    

    def mutate_chromosome(self, chromosome, mutation_rate):
        
        #Swap two trips
        #Move a product to another trip
        #Remove a product from a trip
        
        mutated_chromosome = list(chromosome)
    
        for i in range(len(mutated_chromosome)):
            if random.random() < mutation_rate:
                strategy = random.choice(['swap_trips', 'move_product', 'remove_product'])
                if strategy == 'swap_trips':
                    j = random.randint(0, len(mutated_chromosome) - 1)
                    while j == i:
                        j = random.randint(0, len(mutated_chromosome) - 1)
                    mutated_chromosome[i], mutated_chromosome[j] = mutated_chromosome[j], mutated_chromosome[i]
            
                elif strategy == 'move_product':
                    if mutated_chromosome[i]:  # Non-empty trip
                        prod_id = random.choice(mutated_chromosome[i])  # Random product of this trip ... 
                        valid_trips = [] # Trips that can accept this product in our chromosome
                        product = None
                        for p in self.products:
                            if p.id == prod_id:
                                    product = p
                                    break
                        for j in range(len(mutated_chromosome)): # Check other chromosome trips to find valid trips for our product
                            if j != i and len(mutated_chromosome[j]) < 2 and product.expiry > j:  # another non-full chromosome trip
                                if not mutated_chromosome[j]:  # Empty trip
                                    valid_trips.append(j)
                                else:
                                    other_prod_id = mutated_chromosome[j][0]
                                    other_prod = None
                                    for p in self.products:
                                        if p.id == other_prod_id:
                                            other_prod = p
                                            break  
                                    if other_prod and product.can_ship_together(other_prod):  
                                        valid_trips.append(j)
                    
                        if valid_trips:
                            # Remove from current trip
                            new_trip_i = list(mutated_chromosome[i])
                            new_trip_i.remove(prod_id)
                            mutated_chromosome[i] = tuple(new_trip_i) if new_trip_i else tuple()
                        
                            # Add to new trip
                            dest_trip = random.choice(valid_trips)
                            new_trip_j = list(mutated_chromosome[dest_trip]) if mutated_chromosome[dest_trip] else []
                            new_trip_j.append(prod_id)
                            mutated_chromosome[dest_trip] = tuple(new_trip_j)
            
                elif strategy == 'remove_product':
                    if mutated_chromosome[i] and len(mutated_chromosome[i]) == 2:
                        new_trip = list(mutated_chromosome[i])
                        new_trip.pop(random.randint(0, 1))
                        mutated_chromosome[i] = tuple(new_trip)

        return mutated_chromosome
    
    def crossover(self, parents):
    
        next_generation = []
        parents_with_fitness = [(parent, self.fitness(parent)) for parent in parents]
        parents_with_fitness.sort(key=lambda x: x[1], reverse=True)
    
    
        elite_count = int(self.population_size * self.elite_ratio)
        next_generation.extend([parent for parent, _ in parents_with_fitness[:elite_count]])
    
        # remaining offspring through crossover...
        attempts_per_pair = 3  
        #  roulette wheel again
        available_fitnesses = [fitness for _, fitness in parents_with_fitness]
        min_fitness = min(available_fitnesses)
        shifted_fitnesses = [f - min_fitness + 1 for f in available_fitnesses]
        total_fitness = sum(shifted_fitnesses)
        available_probs = [f / total_fitness for f in shifted_fitnesses]

        while len(next_generation) < self.population_size:
            valid_child_created = False
            parent1 = random.choices(parents_with_fitness, weights=available_probs, k=1)[0][0]
            parent2 = random.choices(parents_with_fitness, weights=available_probs, k=1)[0][0]

            while parent1 == parent2:
                parent2 = random.choices(parents_with_fitness, weights=available_probs, k=1)[0][0]
           
            # Try to create valid child
            for _ in range(attempts_per_pair):
                if self.fitness(parent1) >= self.fitness(parent2):
                    stronger_parent = parent1
                    weaker_parent = parent2
                else:
                    stronger_parent = parent2
                    weaker_parent = parent1
                
                child = self.create_child(stronger_parent, weaker_parent, self.inheritance_ratio)
                child = self.mutate_chromosome(child, mutation_rate=self.mutation_rate)
                
                if self.is_valid_chromosome(child):
                    next_generation.append(child)
                    valid_child_created = True
                    break


            if not valid_child_created and len(next_generation) < self.population_size:
                next_generation.append(stronger_parent)

        
        return next_generation
    


    def run(self):
        print("\n=== Starting Genetic Algorithm ===\n")
        population = self.generate_population()
        print("Initial Population Analysis:")
        self.analyze_population(population)
        print("\n" + "="*50 + "\n")
        no_improvement_count = 0
        for generation in range(self.generations):
            print(f"\nGeneration {generation + 1}/{self.generations}")
            print("-" * 30)
        
            parents = self.selection(population)
        
            # print("\nSelected Parents for Crossover:")
            # self.analyze_population(parents)
        
            
            population = self.crossover(parents)
            print("\nNew Generation After Crossover and Mutation:")
            self.analyze_population(population)
        
        
            fitness_population = [(self.fitness(chromosome), chromosome) for chromosome in population]
            fitness_population.sort(key=lambda x: x[0], reverse=True)
            current_best_fitness, current_best = fitness_population[0]
        
            if current_best_fitness > self.best_fitness:
                self.best_fitness = current_best_fitness
                self.best_solution = current_best
                self.best_generation = generation + 1
                print(f"\n  |_[+_+]_| New Best Solution Found!")
                print(f"Fitness: {self.best_fitness}")
                print(f"Chromosome: {self.best_solution}")
                print(f"Found in Generation: {self.best_generation}")
            else:
                no_improvement_count += 1
                print(f"\nNo improvement in Generation {generation + 1}. No Improvement Count: {no_improvement_count}")

            if no_improvement_count >= 3:
                print(f"\nNo improvement for {no_improvement_count} generations. Stopping early.")
                break

        
            print("\n" + "="*50 + "\n")
    
    
        print("\n=== Final Results ===")
        print(f"\nBest Solution Found:")
        print(f"Chromosome: {self.best_solution}")
        print(f"Fitness: {self.best_fitness}")
        print(f"Found in Generation: {self.best_generation}")
        print("\nDetailed Analysis of Best Solution:")
        self.analyze_population([self.best_solution])
        
        
        
        
        

In [70]:
products, MAX_TRIPS , population_size, parents_ratio , elite_ratio , mutation_rate , inheritance_ratio, generations  = init()

In [71]:
display_products_table(products)

,ID,Expiry,Value,Quantity,Incompatible
0,1,3,15,4,"1, 10"
1,2,6,10,2,"4, 5"
2,3,5,30,10,
3,4,4,25,4,7
4,5,9,20,3,6
5,6,1,50,4,1
6,7,7,35,5,"5, 10"
7,8,8,25,1,"1, 8"
8,9,5,50,3,"2, 3"
9,10,5,20,3,"1, 5"


In [72]:
ga = GeneticAlgorithm(products, MAX_TRIPS, population_size, parents_ratio , elite_ratio , mutation_rate , inheritance_ratio , generations)
ga.run()


=== Starting Genetic Algorithm ===

Initial Population Analysis:


,chromosome,Fitness,Valid,Trips Used,P1%,P2%,P3%,P4%,P5%,P6%,P7%,P8%,P9%,P10%
0,"[(3, 3), (10, 3), (2, 3), (7, 9), (5, 3), (7, 2), (5, 8), (5,), (), (), (), (), (), (), ()]",-360,True,8,0%,100%,50%,0%,100%,0%,40%,100%,33%,33%
1,"[(3, 7), (1, 4), (3, 3), (10, 3), (3, 3), (2, 7), (7, 8), (5, 5), (5,), (), (), (), (), (), ()]",-270,True,9,25%,50%,60%,25%,100%,0%,60%,100%,0%,33%
2,"[(1, 5), (5, 3), (7, 7), (3, 3), (7, 9), (7, 8), (7,), (5,), (), (), (), (), (), (), ()]",-320,True,8,25%,0%,30%,0%,100%,0%,100%,100%,33%,0%
3,"[(6, 3), (7, 1), (3, 3), (3, 4), (7, 8), (5, 5), (7, 7), (5,), (), (), (), (), (), (), ()]",-280,True,8,25%,0%,40%,25%,100%,25%,80%,100%,0%,0%
4,"[(5, 4), (3, 7), (5, 4), (7, 3), (8, 9), (2, 7), (5,), (), (), (), (), (), (), (), ()]",-430,True,7,0%,50%,20%,50%,100%,0%,60%,100%,33%,0%
5,"[(4, 3), (7, 3), (9, 1), (3, 3), (10, 3), (7, 7), (7, 8), (5, 5), (5,), (), (), (), (), (), ()]",-180,True,9,25%,0%,50%,25%,100%,0%,80%,100%,33%,33%
6,"[(4, 3), (1, 7), (8, 3), (3, 7), (9, 10), (7, 2), (5, 5), (5,), (), (), (), (), (), (), ()]",-350,True,8,25%,50%,30%,25%,100%,0%,60%,100%,33%,33%
7,"[(3, 5), (3, 5), (4, 1), (10, 3), (7, 9), (7, 2), (7, 8), (5,), (), (), (), (), (), (), ()]",-350,True,8,25%,50%,30%,25%,100%,0%,60%,100%,33%,33%
8,"[(7, 9), (2, 1), (3, 3), (7, 3), (7, 3), (8, 5), (5, 5), (), (), (), (), (), (), (), ()]",-380,True,7,25%,50%,40%,0%,100%,0%,60%,100%,33%,0%
9,"[(1, 3), (2, 7), (5, 3), (10, 3), (3, 7), (2, 7), (5, 5), (8,), (), (), (), (), (), (), ()]",-420,True,8,25%,100%,40%,0%,100%,0%,60%,100%,0%,33%





Generation 1/10
------------------------------

New Generation After Crossover and Mutation:


,chromosome,Fitness,Valid,Trips Used,P1%,P2%,P3%,P4%,P5%,P6%,P7%,P8%,P9%,P10%
0,"[(9, 4), (3, 3), (1, 7), (4, 3), (10, 3), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",-170,True,9,25%,0%,40%,50%,100%,0%,100%,0%,33%,33%
1,"[(4, 3), (7, 3), (9, 1), (3, 3), (10, 3), (7, 7), (7, 8), (5, 5), (5,), (), (), (), (), (), ()]",-180,True,9,25%,0%,50%,25%,100%,0%,80%,100%,33%,33%
2,"[(2, 6), (10, 3), (5, 9), (7, 3), (3, 7), (7, 8), (7, 7), (5, 5), (), (), (), (), (), (), ()]",-190,True,8,0%,50%,30%,0%,100%,25%,100%,100%,33%,33%
3,"[(7, 3), (2, 8), (3, 2), (4, 3), (9, 10), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",-220,True,9,0%,100%,30%,25%,100%,0%,100%,100%,33%,33%
4,"[(1, 5), (7, 7), (5, 8), (9, 4), (9, 9), (7, 2), (7, 7), (5,), (), (), (), (), (), (), ()]",-230,True,8,25%,50%,0%,25%,100%,0%,100%,100%,100%,0%
5,"[(4, 6), (1, 4), (3, 3), (5, 4), (3, 7), (7, 8), (7, 7), (5, 5), (), (), (), (), (), (), ()]",-240,True,8,25%,0%,30%,75%,100%,25%,80%,100%,0%,0%
6,"[(3, 3), (3, 4), (10, 3), (9, 7), (5, 9), (7, 7), (5, 5), (8,), (), (), (), (), (), (), ()]",-240,True,8,0%,0%,40%,25%,100%,0%,60%,100%,67%,33%
7,"[(4, 6), (3, 1), (3, 3), (7, 3), (2, 3), (2, 8), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",-250,True,9,25%,100%,50%,25%,100%,25%,60%,100%,0%,0%
8,"[(4, 9), (3, 7), (7, 3), (10, 3), (7, 3), (2, 7), (5, 8), (5, 5), (), (), (), (), (), (), ()]",-250,True,8,0%,50%,40%,25%,100%,0%,80%,100%,33%,33%
9,"[(3, 7), (1, 4), (3, 3), (10, 3), (3, 3), (2, 7), (7, 8), (5, 5), (5,), (), (), (), (), (), ()]",-270,True,9,25%,50%,60%,25%,100%,0%,60%,100%,0%,33%



  |_[+_+]_| New Best Solution Found!
Fitness: -90
Chromosome: [(4, 6), (7, 7), (3, 3), (9, 4), (9, 9), (7, 8), (7, 7), (5,), (), (), (), (), (), (), ()]
Found in Generation: 1



Generation 2/10
------------------------------

New Generation After Crossover and Mutation:


...

,chromosome,Fitness,Valid,Trips Used,P1%,P2%,P3%,P4%,P5%,P6%,P7%,P8%,P9%,P10%
0,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
1,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
2,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
3,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
4,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
5,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
6,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
7,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
8,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
9,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%



No improvement in Generation 7. No Improvement Count: 2



Generation 8/10
------------------------------

New Generation After Crossover and Mutation:


,chromosome,Fitness,Valid,Trips Used,P1%,P2%,P3%,P4%,P5%,P6%,P7%,P8%,P9%,P10%
0,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
1,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
2,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
3,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
4,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
5,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
6,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
7,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
8,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
9,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%



No improvement in Generation 8. No Improvement Count: 3

No improvement for 3 generations. Stopping early.

=== Final Results ===

Best Solution Found:
Chromosome: [(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]
Fitness: 0
Found in Generation: 5

Detailed Analysis of Best Solution:


,chromosome,Fitness,Valid,Trips Used,P1%,P2%,P3%,P4%,P5%,P6%,P7%,P8%,P9%,P10%
0,"[(4, 6), (7, 3), (3, 3), (9, 4), (9, 9), (7, 7), (7, 7), (5, 5), (5,), (), (), (), (), (), ()]",0,True,9,0%,0%,30%,50%,100%,25%,100%,0%,100%,0%
0,Summary,Best: 0,Valid: 1/1,Avg: 9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
